In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt  
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import datetime
import torch.utils.data as Data 
torch.manual_seed(5)

#参数

D = 300 
C = 5
Ci=1
len_sentence=57
dropout_rate=0.5 
kernel_size_x1=3
kernel_size_y1=4 
kernel_size1=(kernel_size_x1,kernel_size_y1) 
batch_size1=len_sentence-kernel_size_x1+1 
Kys=100
Co=2

class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()

        # print(args)
        self.hidden_dim =300 #(300-Kys+1)*Co
        self.num_layers =1



        '''
        #BiLSTM
        self.bilstm = nn.LSTM(D_lstm, self.hidden_dim, num_layers=self.num_layers, dropout=dropout_rate, bidirectional=True, bias=True)
        print(self.bilstm)
        self.hidden2label = nn.Linear(self.hidden_dim * 2, C)
        '''

        self.conv1 = nn.Conv2d(Ci, Co, (kernel_size_x1, Kys))

        self.lstm1 = nn.LSTM((300-Kys+1)*Co, self.hidden_dim, dropout=dropout_rate, num_layers=self.num_layers)
        # linear
        self.hidden2label1 = nn.Linear(self.hidden_dim, C)
        # hidden
        self.hidden1 = self.init_hidden(self.num_layers, batch_size1)
        # dropout
        self.batchnorm=nn.BatchNorm2d(1)

    def init_hidden(self, num_layers, batch_size):
        # the first is the hidden h
        # the second is the cell  c
         return (Variable(torch.zeros(1*num_layers, batch_size, self.hidden_dim)).cuda(),
                  Variable(torch.zeros(1*num_layers, batch_size, self.hidden_dim)).cuda())
        #return (Variable(torch.randn(1 * num_layers, batch_size, self.hidden_dim)),
        #         Variable(torch.randn(1 * num_layers, batch_size, self.hidden_dim)))


    def forward(self, x):
        x = x.unsqueeze(1) # (N,Ci,W,D)
        #print x.size()
        
        x1=self.conv1(x)#remove pooling
        y1=x1[:,0]

        for i in range(x1.size()[1]-1):
            y1=torch.cat((y1,x1[:,i+1]),2)
        x1=y1
        x1 = x1.view(len(x1), x1.size(1), -1)
        
        x1 = x1.unsqueeze(1)
        x1=self.batchnorm(x1)
        x1 = x1.squeeze(1)


        lstm_out1, self.hidden1 = self.lstm1(x1, self.hidden1)

        
        lstm_out1 = torch.transpose(lstm_out1, 1, 2)

        # pooling
        lstm_out1 = F.max_pool1d(lstm_out1, lstm_out1.size(2)).squeeze(2)

        lstm_out1 = F.tanh(lstm_out1)
        # linear
        y1 = self.hidden2label1(lstm_out1)
        logit=y1
        return logit

net = LSTM() 
net=net.cuda()
print(net)


LSTM(
  (conv1): Conv2d(1, 2, kernel_size=(3, 100), stride=(1, 1))
  (lstm1): LSTM(402, 300, dropout=0.5)
  (hidden2label1): Linear(in_features=300, out_features=5, bias=True)
  (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True)
)
